In [13]:
import pandas as pd
import numpy as np

from scipy.stats import gamma

from datetime import datetime as dt
import json

In [2]:
confirmed_data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [3]:
confirmed_data = (confirmed_data.groupby('Country/Region').sum()).drop(columns=['Lat','Long'])

In [4]:
data_dates = pd.to_datetime(confirmed_data.columns, format='%m/%d/%y') # $dates
countries_list = list(confirmed_data.index)

In [5]:
confirmed_data = np.array(confirmed_data)

In [6]:
epicurves = np.diff(confirmed_data, n=1, axis=1)
epicurves[epicurves < 0] = 0

In [7]:
def all_nonzero_window_starts(epicurve, win_start, win_size):
    win_end = epicurve.shape[0]
    all_win_starts = list()
    for t in range(win_start,(win_end - win_size)+1):
        if np.sum(epicurve[t:t+win_size+1]) > 0:
            all_win_starts.append(t)
    return all_win_starts


def p_all_taus(epicurve, theta):
    n_observations = epicurve.shape[0]
    all_taus = np.subtract.outer(range(n_observations), range(n_observations))
    w_all_taus = gamma.pdf(all_taus, a=theta[0], scale=theta[1])
    sum_w_all_taus = np.sum(w_all_taus*epicurve.reshape(1,-1), axis=1)
    sum_w_all_taus_matrix = np.tile(sum_w_all_taus, reps=(epicurve.shape[0],1))
    sum_w_all_taus_matrix = np.transpose(sum_w_all_taus_matrix)
    
    p = np.true_divide(w_all_taus, sum_w_all_taus_matrix)
    p[np.isinf(p)] = 0.0
    p[np.isnan(p)] = 0.0
    return p


def mean_r_all_windows(epicurve, mean_r_per_day, win_starts, win_size):
    n_windows = len(win_starts)
    mean_r_per_window = np.zeros(n_windows, np.float32)
    for i in range(n_windows):
        idx = win_starts[i]
        idx_end = idx + win_size + 1
        mean_r_per_window[i] = np.sum((mean_r_per_day[idx:idx_end]*epicurve[idx:idx_end]))/np.sum(epicurve[idx:idx_end])
    return mean_r_per_window


def compute_dynamic_r(epicurves, epi_dates, countries_list):
    dynamic_r = dict() # keys are countries
    win_size = 7
    
    # gamma serial interval distribution parameters
    theta = [1.46, 0.78]
    
    # for each region, assure at least one case exists before the window begins
    window_starts_per_region = np.argmax(epicurves>0, axis=1) + 1
    for idx in range(epicurves.shape[0]):
        region_dynamic_r = dict() # keys are 'dates', 'mean_r'
        
        epicurve = epicurves[idx]
        all_win_starts = all_nonzero_window_starts(epicurve, 
                                                   window_starts_per_region[idx], 
                                                   win_size = win_size)
        p = p_all_taus(epicurve, theta)
        mean_r_per_day = np.sum(p*epicurve.reshape(-1,1), axis=0)
        mean_r_per_window = mean_r_all_windows(epicurve, mean_r_per_day, 
                                               all_win_starts, win_size)
        
        region_dynamic_r['dates'] = epi_dates[all_win_starts]
        region_dynamic_r['mean_r'] = mean_r_per_window.tolist()
        dynamic_r[countries_list[idx]] = region_dynamic_r
    
    return dynamic_r


In [39]:
rnumber = compute_dynamic_r(epicurves, data_dates, countries_list)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [40]:
rnumber['US']['mean_r']

[0.997429609298706,
 0.997429609298706,
 0.8316113352775574,
 0.8316113352775574,
 0.8316113352775574,
 0.8316113352775574,
 0.8316113352775574,
 0.694835901260376,
 0.31906798481941223,
 0.31906798481941223,
 1.1570727825164795,
 1.1570727825164795,
 1.5004961490631104,
 1.5004961490631104,
 1.5004961490631104,
 1.5004961490631104,
 1.5004961490631104,
 1.5004961490631104,
 1.8439195156097412,
 1.8439195156097412,
 18.006494522094727,
 18.006494522094727,
 18.006494522094727,
 1.1644078493118286,
 1.1644078493118286,
 1.0876164436340332,
 1.0961300134658813,
 1.170563817024231,
 0.6263543963432312,
 0.8702800869941711,
 1.0161212682724,
 1.67244291305542,
 1.8084689378738403,
 1.673782229423523,
 1.8090848922729492,
 1.5727312564849854,
 1.5797991752624512,
 1.7873071432113647,
 1.5566165447235107,
 1.48228120803833,
 1.448945164680481,
 1.4187699556350708,
 1.435773491859436,
 1.5111576318740845,
 1.5820051431655884,
 1.6106138229370117,
 1.9295324087142944,
 1.6342310905456543,
 1.5

In [41]:
for country in rnumber.keys():
    rnumber[country]['dates'] = list(np.datetime_as_string(rnumber[country]['dates'], unit='D'))

In [43]:
with open('/Users/balasubramanijb/Documents/projects/covid_19/rnumber/data/covid_effective_r.json','w') as json_file:
    json.dump(rnumber, json_file, sort_keys=True, indent=4, separators=(',', ': '))